In [62]:
import pandas as pd
from pyspark.sql import SparkSession, functions as F
import lbl2vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
from pyspark.sql.functions import date_format
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import OneHotEncoder, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt


In [4]:
# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "10g")
    .getOrCreate()
)

22/10/03 22:24:17 WARN Utils: Your hostname, MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.0.66 instead (on interface en0)
22/10/03 22:24:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/03 22:24:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/03 22:24:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/03 22:24:18 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
# Read in data from ETL.py file
%run '../scripts/ETL.py' '../scripts/paths.json'
final_join3.limit(5)

/Users/Kasturi/opt/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


22/10/03 22:24:20 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


22/10/03 22:25:13 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


merchant_name,merchant_abn,categories,take_rate,revenue_levels,name,address,state,gender,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id,postcodes,int_sa2,SA2_code,SA2_name,income_2018-2019,total_males,total_females,total_persons,state_code,state_name,population_2020,population_2021
Egestas Nunc Asso...,11121775571,digital goods: bo...,6.58,a,Christopher Rodri...,30554 Evans Strea...,NSW,Male,11121775571,11.28829564583802,2bd2a61d-72e5-42d...,2021-08-20,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694
Morbi Accumsan In...,19618998054,tent and aWning s...,1.52,c,Christopher Rodri...,30554 Evans Strea...,NSW,Male,19618998054,62.90176609196828,3582b1f8-4577-403...,2021-05-16,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694
Eu Dolor Egestas PC,94472466107,"cable, satellite,...",6.23,a,Christopher Rodri...,30554 Evans Strea...,NSW,Male,94472466107,172.15375126873164,cb05d49f-c2fa-453...,2021-07-22,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694
Urna Justo Indust...,31472801314,music shops - mus...,6.56,a,Christopher Rodri...,30554 Evans Strea...,NSW,Male,31472801314,0.4894787650356477,aeec15c1-67e8-4cb...,2021-05-18,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694
Eu Sem Pellentesq...,35424691626,"computers, comput...",3.9,b,Christopher Rodri...,30554 Evans Strea...,NSW,Male,35424691626,7.360217018778133,9df473ba-102d-461...,2021-07-04,3698,1175,2299,111031231,111031231,Shortland - Jesmond,242936885,6412,6179,12593,1,New South Wales,12598,12694


take rate
state
gender
year
month
day
postcode
merchant name



In [185]:
final_join3.createOrReplaceTempView("temp")

d = spark.sql(""" 

SELECT COUNT(DISTINCT take_rate), merchant_name
FROM temp
GROUP BY merchant_name
""")

d.show()

+-------------------------+--------------------+
|count(DISTINCT take_rate)|       merchant_name|
+-------------------------+--------------------+
|                        1|     Erat Semper Ltd|
|                        1|Dictum Mi Incorpo...|
|                        1|At Augue Corporation|
|                        1|Porttitor Eros In...|
|                        1|Mollis Integer Co...|
|                        1|Varius Orci Insti...|
|                        1|Volutpat Nulla In...|
|                        1|      Amet Nulla Ltd|
|                        1|Hendrerit Consect...|
|                        1|   Malesuada Vel Ltd|
|                        1|  Vel Turpis Company|
|                        1|Elit Sed Consequa...|
|                        1|        Malesuada PC|
|                        1|   Pede Praesent Ltd|
|                        1|   Dictum Mi Limited|
|                        1|Vulputate Velit E...|
|                        1| Phasellus Nulla LLC|
|                   

In [25]:
merchants = spark.read.parquet("/Users/Kasturi/Documents/GitHub/generic-buy-now-pay-later-project-group-10-bnpl/data/tables/tbl_merchants.parquet")

In [26]:
merchants.show(5)

+--------------------+--------------------+------------+
|                name|                tags|merchant_abn|
+--------------------+--------------------+------------+
|       Felis Limited|((furniture, home...| 10023283211|
|Arcu Ac Orci Corp...|([cable, satellit...| 10142254217|
|    Nunc Sed Company|([jewelry, watch,...| 10165489824|
|Ultricies Digniss...|([wAtch, clock, a...| 10187291046|
| Enim Condimentum PC|([music shops - m...| 10192359162|
+--------------------+--------------------+------------+
only showing top 5 rows



In [6]:
final_join3.count()

10540181

In [140]:
tagged_merchants = pd.read_csv("../data/curated/tagged_merchants.csv")

In [141]:
tagged_merchants = tagged_merchants.iloc[:,1:]

In [142]:
tagged_merchants.drop(['tags', 'name', 'cleaned_tags', 'store_type'], axis=1, inplace=True)

In [143]:
tagged_merchants.to_parquet("../data/curated/tagged_merchants.parquet")

In [144]:
tagged_merchants_sdf = spark.read.parquet("../data/curated/tagged_merchants.parquet")

In [145]:
tagged_merchants_sdf = tagged_merchants_sdf.withColumnRenamed('merchant_abn',

    'tagged_merchant_abn'
)

In [146]:
tagged_merchants_sdf.show(5)

+-------------------+--------------------+
|tagged_merchant_abn|            category|
+-------------------+--------------------+
|        10023283211|           Furniture|
|        10142254217|         Electronics|
|        10165489824|        Toys and DIY|
|        10187291046|        Toys and DIY|
|        10192359162|Books, Stationary...|
+-------------------+--------------------+
only showing top 5 rows



In [147]:
final_join3.createOrReplaceTempView("join")
tagged_merchants_sdf.createOrReplaceTempView("tagged")

joint = spark.sql(""" 

SELECT *
FROM join
INNER JOIN tagged
ON join.merchant_abn = tagged.tagged_merchant_abn
""")



In [148]:
joint = joint.drop('tagged_merchant_abn')

In [149]:
joint.count()

10109254

In [150]:
joint.createOrReplaceTempView("group")

a = spark.sql(""" 

SELECT *, (dollar_value - take_rate) AS total_earning
FROM group
""")

In [151]:
a.createOrReplaceTempView("group_earnings")

b = spark.sql(""" 

SELECT merchant_name, SUM(total_earning) AS total_revenue, COUNT(merchant_abn) AS no_of_transactions
FROM group_earnings
GROUP BY merchant_name
""")

b.show()

+--------------------+------------------+------------------+
|       merchant_name|     total_revenue|no_of_transactions|
+--------------------+------------------+------------------+
|   Dictum Mi Limited| 897687.9661494899|              9880|
|Volutpat Nulla In...| 836578.8864434501|             10561|
|Mollis Integer Co...|384905.39521080774|             12234|
|Hendrerit Consect...|239274.53116224083|              3010|
|Donec Luctus Indu...|  1063577.03533018|              3113|
|Varius Orci Insti...|147376.27431885863|             19771|
|Elit Sed Consequa...| 5724004.642736054|              9689|
|Dictum Mi Incorpo...| 775536.8639499726|              1236|
|At Augue Corporation|125257.92718248101|              1544|
|      Montes Limited|429378.17366086063|               673|
|     Erat Semper Ltd| 639325.5782926289|              1781|
| Semper Incorporated| 92248.05999937073|               926|
|Porttitor Eros In...|121304.87964357901|              1262|
|   Malesuada Vel Ltd| 1

In [152]:
# Extracting the year, month, day from the timestamp

a = a.withColumn("Year", 
date_format('order_datetime', 'yyyy'))

a  = a.withColumn("Month", 
date_format('order_datetime', 'MMMM'))


a = a.withColumn("Day",
date_format(("order_datetime"), "E"))


In [153]:
a.show(5)

+--------------------+------------+--------------------+---------+--------------+--------------------+--------------------+-----+------+------------------+------------------+--------------------+--------------+-------+-----------+---------+---------+---------+-------------------+----------------+-----------+-------------+-------------+----------+---------------+---------------+---------------+--------------------+------------------+----+------+---+
|       merchant_name|merchant_abn|          categories|take_rate|revenue_levels|                name|             address|state|gender|trans_merchant_abn|      dollar_value|            order_id|order_datetime|user_id|consumer_id|postcodes|  int_sa2| SA2_code|           SA2_name|income_2018-2019|total_males|total_females|total_persons|state_code|     state_name|population_2020|population_2021|            category|     total_earning|Year| Month|Day|
+--------------------+------------+--------------------+---------+--------------+-------------

In [154]:
train_data = a.drop('merchant_abn', 'categories','name', 'address', 'trans_merchant_abn', 'order_id','order_datetime','user_id','consumer_id','int_sa2','SA2_code',
'SA2_name','income_2018-2019','total_males', 'total_females','total_persons', 'state_code','state_name','population_2020', 'population_2021','total_earning')

In [155]:
 
# Find Count of Null, None, NaN of All DataFrame Columns
from pyspark.sql.functions import col,isnan, when, count
train_data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in train_data.columns]
   ).show()

+-------------+---------+--------------+-----+------+------------+---------+--------+----+-----+---+
|merchant_name|take_rate|revenue_levels|state|gender|dollar_value|postcodes|category|Year|Month|Day|
+-------------+---------+--------------+-----+------+------------+---------+--------+----+-----+---+
|            0|        0|             0|    0|     0|           0|        0|       0|   0|    0|  0|
+-------------+---------+--------------+-----+------+------------+---------+--------+----+-----+---+



In [156]:
train_data.printSchema()

root
 |-- merchant_name: string (nullable = true)
 |-- take_rate: double (nullable = true)
 |-- revenue_levels: string (nullable = true)
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- dollar_value: double (nullable = true)
 |-- postcodes: string (nullable = true)
 |-- category: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Day: string (nullable = true)



In [160]:
a.createOrReplaceTempView("group_earnings")

e = spark.sql(""" 

SELECT COUNT(DISTINCT state)
FROM group_earnings
GROUP BY merchant_name
""")

e.show()

+---------------------+
|count(DISTINCT state)|
+---------------------+
|                    7|
|                    7|
|                    7|
|                    7|
|                    7|
|                    7|
|                    7|
|                    7|
|                    7|
|                    7|
|                    7|
|                    6|
|                    7|
|                    7|
|                    5|
|                    7|
|                    7|
|                    7|
|                    7|
|                    7|
+---------------------+
only showing top 20 rows



In [ ]:

merchant name
take rate
revenue levels
number of transactions
year
month
sum(dollar value)
numer of males
number of females
category


aggregate merchant name by year, month



In [166]:
a.createOrReplaceTempView("agg")

male = spark.sql(""" 

SELECT merchant_name AS m_name, COUNT(gender) as males
FROM agg
WHERE gender = 'Male'
GROUP BY merchant_name
""")

male.show(5)

female = spark.sql(""" 

SELECT merchant_name AS f_name, COUNT(gender) as females
FROM agg
WHERE gender = 'Female'
GROUP BY merchant_name
""")
female.show(5)

+--------------------+-----+
|              m_name|males|
+--------------------+-----+
|Elit Sed Consequa...| 4376|
|Varius Orci Insti...| 8966|
|Mollis Integer Co...| 5507|
|Donec Luctus Indu...| 1367|
|Volutpat Nulla In...| 4730|
+--------------------+-----+
only showing top 5 rows



+--------------------+-------+
|              f_name|females|
+--------------------+-------+
|Varius Orci Insti...|   8743|
|Volutpat Nulla In...|   4711|
|Donec Luctus Indu...|   1437|
|Mollis Integer Co...|   5456|
|      Montes Limited|    310|
+--------------------+-------+
only showing top 5 rows



In [168]:
a.createOrReplaceTempView("agg")

temp = spark.sql(""" 

SELECT merchant_name, COUNT(merchant_name) AS no_of_transactions, Year, Month, SUM(dollar_value) AS total_earnings
FROM group_earnings
GROUP BY merchant_name, Year, Month
""")

temp.show()


+--------------------+------------------+----+------+------------------+
|       merchant_name|no_of_transactions|Year| Month|    total_earnings|
+--------------------+------------------+----+------+------------------+
|Risus Nunc Ac Ins...|                27|2021|  July| 45120.95508454241|
|  Euismod Urna Corp.|                69|2021|August| 8312.634855809609|
|Ipsum Cursus Vest...|               132|2021|August|11840.789248687695|
|Feugiat Metus Sit...|               267|2021|   May|40941.767714156485|
|  Aliquam Gravida PC|               159|2021|  July| 38688.69671486665|
|Ridiculus Mus Aen...|                94|2021|  June|5266.7737808939755|
|Quis Pede Corpora...|                28|2021|  July| 4980.521001033796|
|      Nisi Cum Corp.|                53|2021|  July| 33675.74676686274|
|     Vivamus Sit LLC|               139|2021|   May| 36767.03365261648|
|Ipsum Primis In Inc.|               174|2021|   May|27242.934114830754|
|       Vulputate LLC|               180|2021|   Ma

In [170]:
temp.createOrReplaceTempView("gender_join")
male.createOrReplaceTempView("m")
female.createOrReplaceTempView("f")

temp2 = spark.sql(""" 

SELECT *
FROM gender_join
INNER JOIN m
ON gender_join.merchant_name = m.m_name
""")

temp2.createOrReplaceTempView("temp2")

temp3 = spark.sql(""" 

SELECT *
FROM temp2
INNER JOIN f
ON temp2.merchant_name = f.f_name
""")

temp3.limit(5)

merchant_name,no_of_transactions,Year,Month,total_earnings,m_name,males,f_name,females
Elit Sed Consequa...,372,2022,February,235567.82708092846,Elit Sed Consequa...,4376,Elit Sed Consequa...,4331
Elit Sed Consequa...,485,2022,May,286509.54539073445,Elit Sed Consequa...,4376,Elit Sed Consequa...,4331
Elit Sed Consequa...,366,2022,January,226384.3371305042,Elit Sed Consequa...,4376,Elit Sed Consequa...,4331
Elit Sed Consequa...,13,2021,February,7390.9650375502115,Elit Sed Consequa...,4376,Elit Sed Consequa...,4331
Elit Sed Consequa...,475,2022,October,285786.0782532809,Elit Sed Consequa...,4376,Elit Sed Consequa...,4331


In [188]:
a.createOrReplaceTempView("features")

e = spark.sql(""" 

SELECT merchant_name AS drop_name, FIRST(take_rate) AS take_rate, FIRST(revenue_levels) AS revenue_levels
FROM features
GROUP BY merchant_name
""")

e.show(2)

+--------------------+---------+--------------+
|           drop_name|take_rate|revenue_levels|
+--------------------+---------+--------------+
|A Arcu Sed Corpor...|     3.06|             c|
|        A Associates|     4.95|             b|
+--------------------+---------+--------------+
only showing top 2 rows



In [189]:
temp3.createOrReplaceTempView("edit")
e.createOrReplaceTempView("rates")

temp4 = spark.sql(""" 

SELECT *
FROM edit
INNER JOIN rates
ON edit.merchant_name = rates.drop_name
""")

train = temp4.drop('m_name', 'f_name', 'drop_name')

train.limit(5)

merchant_name,no_of_transactions,Year,Month,total_earnings,males,females,take_rate,revenue_levels
Elit Sed Consequa...,372,2022,February,235567.82708092846,4376,4331,5.89,a
Elit Sed Consequa...,485,2022,May,286509.54539073445,4376,4331,5.89,a
Elit Sed Consequa...,366,2022,January,226384.3371305042,4376,4331,5.89,a
Elit Sed Consequa...,13,2021,February,7390.9650375502115,4376,4331,5.89,a
Elit Sed Consequa...,475,2022,October,285786.0782532809,4376,4331,5.89,a


In [190]:
train.printSchema()

root
 |-- merchant_name: string (nullable = true)
 |-- no_of_transactions: long (nullable = false)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- total_earnings: double (nullable = true)
 |-- males: long (nullable = false)
 |-- females: long (nullable = false)
 |-- take_rate: double (nullable = true)
 |-- revenue_levels: string (nullable = true)



In [191]:
# String indexing the categorical columns

indexer = StringIndexer(inputCols = ['merchant_name', 'Year', 'Month', 'revenue_levels'],
outputCols = ['merchant_name_num', 'Year_num', 'Month_num', 'revenue_levels_num'])

indexd_data = indexer.fit(train).transform(train)


# Applying onehot encoding to the categorical data that is string indexed above
encoder = OneHotEncoder(inputCols = ['merchant_name_num', 'Year_num', 'Month_num', 'revenue_levels_num'],
outputCols = ['merchant_name_vec', 'Year_vec', 'Month_vec', 'revenue_levels_vec'])

onehotdata = encoder.fit(indexd_data).transform(indexd_data)


# Assembling the training data as a vector of features 
assembler1 = VectorAssembler(
inputCols=['no_of_transactions','males', 'females', 'take_rate','merchant_name_vec', 'Year_vec', 'Month_vec', 'revenue_levels_vec'],
outputCol= "features" )

outdata1 = assembler1.transform(onehotdata)

In [192]:
# Renaming the target column as label

outdata1 = outdata1.withColumnRenamed(
    "total_earnings",
    "label"
)

In [193]:
# Assembling the features as a feature vector 

featureIndexer =\
    VectorIndexer(inputCol="features", 
    outputCol="indexedFeatures").fit(outdata1)

outdata1 = featureIndexer.transform(outdata1)

In [194]:
# Split the data into training and validation sets (30% held out for testing)

trainingData, testData = outdata1.randomSplit([0.7, 0.3], seed = 20)

In [195]:
trainingData.count(), testData.count()

22/10/04 01:10:17 WARN DAGScheduler: Broadcasting large task binary with size 1616.9 KiB


22/10/04 01:10:45 WARN DAGScheduler: Broadcasting large task binary with size 1612.1 KiB


(52487, 22575)

In [196]:
# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="indexedFeatures")


# Train model.  
model = rf.fit(trainingData)

# Make predictions.
predictions_validation = model.transform(testData)

22/10/04 01:11:19 WARN DAGScheduler: Broadcasting large task binary with size 1623.9 KiB


22/10/04 01:11:20 WARN DAGScheduler: Broadcasting large task binary with size 1624.0 KiB


22/10/04 01:11:22 WARN DAGScheduler: Broadcasting large task binary with size 1628.0 KiB


22/10/04 01:11:23 WARN DAGScheduler: Broadcasting large task binary with size 1842.1 KiB


22/10/04 01:11:28 WARN DAGScheduler: Broadcasting large task binary with size 1953.2 KiB


22/10/04 01:11:31 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB


22/10/04 01:11:36 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB


22/10/04 01:11:41 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB


In [197]:
# Evaluate the validation set 

predictions_validation.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error

evaluator_train_rmse = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse_train = evaluator_train_rmse.evaluate(predictions_validation)
print("Root Mean Squared Error (RMSE) on train data = %g" % rmse_train)

evaluator_train_mae = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="mae")
mae_train = evaluator_train_mae.evaluate(predictions_validation)
print("Root Mean Squared Error (MAE) on raint data = %g" % mae_train)

22/10/04 01:12:10 WARN DAGScheduler: Broadcasting large task binary with size 1631.1 KiB


+------------------+------------------+--------------------+
|        prediction|             label|            features|
+------------------+------------------+--------------------+
|14754.095999829009| 5668.045970027864|(3994,[0,1,2,3,16...|
| 17312.92613445704|11213.777401669195|(3994,[0,1,2,3,16...|
| 17312.92613445704| 8485.073354261562|(3994,[0,1,2,3,16...|
| 17312.92613445704|10327.396490189016|(3994,[0,1,2,3,16...|
| 24160.39955276889|33955.585911857626|(3994,[0,1,2,3,4,...|
+------------------+------------------+--------------------+
only showing top 5 rows



22/10/04 01:12:30 WARN DAGScheduler: Broadcasting large task binary with size 1627.3 KiB


Root Mean Squared Error (RMSE) on train data = 34148.1


22/10/04 01:12:51 WARN DAGScheduler: Broadcasting large task binary with size 1627.3 KiB


Root Mean Squared Error (MAE) on raint data = 15655.2
